In [1]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import re

In [5]:
df_degra = pd.read_csv(r'http://sc-e.fr/docs/degradations_2024-06-06.csv')
df_degra

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,54.443290,2024-06-06,False
1,D009402,E170_6353,REAE170-E170_6353-1,51.297412,2024-06-06,False
2,D000812,E170_6353,SYSE170-E170_6353-2,17.647642,2024-06-06,False
3,D006665,E170_6353,ORDE170-E170_6353-3,7.561442,2024-06-06,False
4,D001163,E170_6353,AUTE170-E170_6353-4,43.325214,2024-06-06,False
...,...,...,...,...,...,...
10525,D006322,CRJ700_2988,SYSCRJ700-CRJ700_2988-40,53.688232,2024-06-05,False
10526,D005497,CRJ700_2988,PORCRJ700-CRJ700_2988-41,16.759548,2024-06-05,False
10527,D005625,CRJ700_2988,HUBCRJ700-CRJ700_2988-42,36.482392,2024-06-05,False
10528,D003657,CRJ700_2988,ECLCRJ700-CRJ700_2988-43,37.004796,2024-06-05,False


In [3]:
# date du jour
date_today = datetime.today().strftime('%Y-%m-%d')

In [2]:
date_defini = '2024-06-13'

In [3]:
# Fonction pour supprimer l'heure, les minutes et les secondes représentant une date et une heure
def supprimer_heure_minutes_secondes(chaine):                   
    nouvelle_chaine = re.sub(r'\s\d{2}:\d{2}:\d{2}$', '', chaine)           
    return nouvelle_chaine

In [6]:
# Appliquer la fonction à la colonne 'last_maint'
df_degra['measure_day'] = df_degra['measure_day'].astype(str).apply(supprimer_heure_minutes_secondes)

In [7]:
# Mettre la colonne measure_day au format date
df_degra['measure_day'] = pd.to_datetime(df_degra['measure_day'], format='%Y-%m-%d', errors='coerce')

In [8]:
# Fonction pour ne mettre dans la base de données uniquement les measure_day du jour
def measure_today(data, date):
    filtered_data = data.query('measure_day == @date')
    return filtered_data

In [9]:
# Fonction pour récupérer les données degradations et les stocker dans la base de données SQL
def degradations(date, engine):
    url = f"http://sc-e.fr/docs/degradations_{date}.csv"

    response = requests.get(url)

    data = pd.read_csv(url)

    # Filtrer les données pour ne conserver que celles de la date actuelle
    filtered_data = measure_today(data, date)

    # Stocker les données dans la base de données SQL [name = nom de la table / con = connexion à db / if_exists = ajoute les données]
    filtered_data.to_sql(name='degradations', con=engine, if_exists='append', index=False)

# Configurer la connexion à la base de données
db_path = r'C:\Document\Python\Projet\Projet_3\Top_gun\bdd_sql\top_gun_data_16-9.db'
engine = create_engine(f'sqlite:///{db_path}')

degradations(date_defini, engine)

In [9]:
df_degra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10530 entries, 0 to 10529
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ref_deg           10530 non-null  object        
 1   linked_aero       10530 non-null  object        
 2   compo_concerned   10530 non-null  object        
 3   usure_nouvelle    10530 non-null  float64       
 4   measure_day       10530 non-null  datetime64[ns]
 5   need_replacement  10530 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), object(3)
memory usage: 421.7+ KB


In [10]:
df_degra['measure_day'].value_counts()

measure_day
2024-06-05    4635
2024-06-06    2610
2024-06-04    1665
2024-06-01     540
2024-06-03     450
2024-06-02     405
2023-08-23     135
2024-01-21      45
2024-05-31      45
Name: count, dtype: int64